<a target="_blank" href="https://colab.research.google.com/github/mcosarinsky/TP-NLP/blob/main/notebooks/llama_qa.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
!git clone https://github.com/mcosarinsky/TP-NLP.git
%cd TP-NLP

Cloning into 'TP-NLP'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 67 (delta 27), reused 39 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (67/67), 26.21 MiB | 18.69 MiB/s, done.
Resolving deltas: 100% (27/27), done.
/content/TP-NLP


In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
import torch
import gc
from transformers import pipeline
import os

#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "true"

# Instruct version of the model
model_id = "meta-llama/Llama-3.2-1B-Instruct"

# Initialize the pipeline for text generation
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

c:\Users\Matias\anaconda3\envs\RCA_v2\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Matias\.cache\huggingface\hub\models--meta-llama--Llama-3.2-1B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [3]:
def retrieve_article(df, article_id):
    article = df[df['id'] == article_id]
    return f"\nDiario:{article['site'].values[0]}\nTitulo:{article['title'].values[0]}\nFecha: {article['published date'].values[0]}\nDescripción: {article['description'].values[0]}\nContenido: {article['text'].values[0]}"

def answer_question(prompt):
    # Define model role
    system_message = {"role": "system", "content": "Eres un asistente especializado en responder preguntas sobre artículos periodísticos en español."}

    # Combine system message with the user message (the question)
    messages = [
        system_message,
        {"role": "user", "content": prompt}
    ]

    # Generate the model's response
    try:
        output = pipe(messages, max_new_tokens=256, num_return_sequences=1, pad_token_id = pipe.tokenizer.eos_token_id,
                    do_sample=False, return_full_text=False, temperature=None, top_p=None)
        return output[0]['generated_text']
    finally:
        gc.collect()
        torch.cuda.empty_cache()

In [8]:
import pandas as pd
from tqdm import tqdm

df_preguntas = pd.read_csv('../data/qa_dataset_v2.csv')
df_articulos = pd.read_csv('../data/articulos.csv')
prompt_inicial = "En base a los siguientes articulos:\n"

for ind, pregunta in tqdm(df_preguntas.iterrows(), total=len(df_preguntas)):
    article_1 = retrieve_article(df_articulos, pregunta['top_1_id'])
    #article_2 = retrieve_article(df_articulos, pregunta['top_2_id'])
    #article_3 = retrieve_article(df_articulos, pregunta['top_3_id'])

    prompt = f"{prompt_inicial}Articulo: {article_1}\n"#Articulo 2: {article_2}\nArticulo 3: {article_1}\n"
    prompt += f"Contesta la siguiente pregunta: {pregunta['Pregunta']}. Responde únicamente a la pregunta, la respuesta debe ser directa y precisa; debe consistir en una única frase breve."

    df_preguntas.loc[ind, 'Respuesta modelo'] = answer_question(prompt)

100%|██████████| 198/198 [17:40<00:00,  5.35s/it] 
